In [1]:
param_date = '20240623'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/3.parttype.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Tab_No.'] = input_['Tab_No.'].astype(int)
input_['GroupName_No.'] = input_['GroupName_No.'].astype(int)
input_['SubGroupName_No.'] = input_['SubGroupName_No.'].astype(int)
input_['PartType_No.'] = input_['PartType_No.'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': f'''{{"jsn":{{"tab":"extras","groupname":"{input_.loc[a, 'GroupName']}","subgroupname":"{input_.loc[a, 'SubGroupName']}","parttype":"{input_.loc[a, 'PartType']}","nodetype":"parttype"}}}}''',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp)

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =

                list_spans = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()') for tbody in list_tbody]
                list_span = ['\n'.join([span.strip() for span in spans]) for spans in list_spans]

                # = = = = = = = = = = = = = = =

                list_spans = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row"]/span[@class="mtf-h-t"]/text()') for tbody in list_tbody]
                list_span_2 = ['\n'.join([span.strip() for span in spans]) for spans in list_spans]

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_essential[")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_parttype = [json_['parttype'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_partkey = [json_['partkey'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_a = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href') for tbody in list_tbody]
                for i in range(len(list_a)):
                    if len(list_a[i]) == 0 or 'www.rockauto.com/en/moreinfo.php' not in list_a[i][0]:
                        list_a[i] = f'https://www.rockauto.com/en/moreinfo.php?pk={list_partkey[i]}&pt={list_parttype[i]}&Lennon=1'
                    else:
                        list_a[i] = list_a[i][0].strip()

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_partnumber = [json_['partnumber'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_catalogname = [json_['catalogname'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_sortgrouptext = [json_['sortgrouptext'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_paramdesc = [json_['paramdesc'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_spans = [tbody.xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()') for tbody in list_tbody]
                list_oe = [';'.join([oe.strip() for oe in span.split(',')]) for span in [','.join(spans) for spans in list_spans]]

                # = = = = = = = = = = = = = = =

                list_srcs = [[] if len(srcs) == 0 else json.loads(srcs[0])['Slots'] for srcs in [tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') for tbody in list_tbody]]

                # = = = = = = = = = = = = = = =

                list_json_src = []
                for srcs in list_srcs:
                    dict_src = {}
                    for i in range(len(srcs)):
                        dict_src[str(i)] = 'https://www.rockauto.com' + srcs[i]['ImageData']['Full'].strip()
                    list_json_src.append(json.dumps(dict_src))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Tab_No.': input_.loc[a, 'Tab_No.'],
                                        'Tab': input_.loc[a, 'Tab'],
                                        'GroupName_No.': input_.loc[a, 'GroupName_No.'],
                                        'GroupName': input_.loc[a, 'GroupName'],
                                        'SubGroupName_No.': input_.loc[a, 'SubGroupName_No.'],
                                        'SubGroupName': input_.loc[a, 'SubGroupName'],
                                        'PartType_No.': input_.loc[a, 'PartType_No.'],
                                        'PartType_Name': input_.loc[a, 'PartType_Name'],
                                        'PartType': input_.loc[a, 'PartType'],
                                        'Tab_Url': input_.loc[a, 'Tab_Url'],
                                        'Part_No.': [i+1 for i in range(len(list_json))],
                                        'SortGroupText': list_sortgrouptext,
                                        'CatalogName': list_catalogname,
                                        'PartNumber': list_partnumber,
                                        'Part_PartType': list_parttype,
                                        'PartKey': list_partkey,
                                        'JOIN_PTPKPNCN': [f'{parttype};{partkey};{partnumber};{catalogname}' for parttype, partkey, partnumber, catalogname in zip(list_parttype, list_partkey, list_partnumber, list_catalogname)],
                                        'ParamDesc': list_paramdesc,
                                        'Json_Vehicle': '',
                                        'Vehicle': '',
                                        'OE': list_oe,
                                        'Note_1': list_span,
                                        'Note_2': list_span_2,
                                        'Url': [f'''https://www.rockauto.com/en/parts/{catalogname},{partnumber},{input_.loc[a, 'PartType_Name']},{part_parttype}''' for catalogname, partnumber, part_parttype in zip(list_catalogname, list_partnumber, list_parttype)],
                                        'Info': list_a,
                                        'Json_Src': list_json_src,
                                        'Json_Specification': '',
                                        'Json_Kit': ''})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Tab_No.': input_.loc[a, 'Tab_No.'],
                                     'Tab': input_.loc[a, 'Tab'],
                                     'GroupName_No.': input_.loc[a, 'GroupName_No.'],
                                     'GroupName': input_.loc[a, 'GroupName'],
                                     'SubGroupName_No.': input_.loc[a, 'SubGroupName_No.'],
                                     'SubGroupName': input_.loc[a, 'SubGroupName'],
                                     'PartType_No.': input_.loc[a, 'PartType_No.'],
                                     'PartType_Name': input_.loc[a, 'PartType_Name'],
                                     'PartType': input_.loc[a, 'PartType'],
                                     'Tab_Url': input_.loc[a, 'Tab_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'GroupName_No.']}. {input_.loc[a, 'GroupName']} > {input_.loc[a, 'SubGroupName_No.']}. {input_.loc[a, 'SubGroupName']} > {input_.loc[a, 'PartType_No.']}. {input_.loc[a, 'PartType_Name']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['Tab_No.', 'GroupName_No.', 'SubGroupName_No.', 'PartType_No.', 'Part_No.'], ascending=[True, True, True, True, True], ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['Tab_No.', 'GroupName_No.', 'SubGroupName_No.', 'PartType_No.'], ascending=[True, True, True, True], ignore_index=True)
    output_error.to_excel(f'./{param_date}/part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1729

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 7. 1 13/64" (30.6mm) Belt
[剩余数量：1709] - [当前时间：10:51:14]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 2. 1" (25.4mm) Belt
[剩余数量：1708] - [当前时间：10:51:14]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 5. 1 5/32" (29.3mm) Belt
[剩余数量：1707] - [当前时间：10:51:14]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 4. 1 1/8" (28.6mm) Belt
[剩余数量：1706] - [当前时间：10:51:14]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 10. 1 9/32" (32.5mm) Belt
[剩余数量：1705] - [当前时间：10:51:14]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 11. 1 5/16" (33.3mm) Belt
[剩余数量：1704] - [当前时间：10:51:15]

[状态：ok，尝试次数：  1] - 1. Apparel & Gifts > 2. RockAuto Gift > 2. Lanyard
[剩余数量：1703] - [当前时间：10:51:15]

[状态：ok，尝试次数：  1] - 1. Apparel & Gifts > 1. RockAuto Apparel > 3. T-Shirt
[剩余数量：1702] - [当前时间：10:51:15]

[状态：ok，尝试次数：  1] - 2. Belt Drive > 1. ATV / UTV Belt > 12. 1 11/32" (34.1mm) Belt
[剩余数量：1701] - [当前时间：10:51:15]

[状态：ok，尝试